<a href="https://colab.research.google.com/github/aAmohammadrezaaA/Retinal-Vessel-Segmentation_A-Computer-Vision-Technique/blob/main/implementation_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tabulate
from tabulate import tabulate
from prettytable import PrettyTable
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import concatenate, MaxPooling2D, DepthwiseConv2D, AveragePooling2D,Conv2DTranspose,Input,Add,Conv2D, BatchNormalization,LeakyReLU, Activation, MaxPool2D, Dropout, Flatten, Dense,UpSampling2D,Concatenate,Softmax, Layer
from tensorflow.keras.models import Model
patch_size=48
patch_num=1500
patch_threshold=25
BATCH_SIZE=64
LR=0.0003
channels=3

input_shape = (patch_size, patch_size, channels)  # Adjusting the shape
batch_size = BATCH_SIZE

# random input tensor for testing
input_tensor = tf.random.normal((batch_size,) + input_shape)

class LinearTransform(tf.keras.Model):
  def __init__(self, name="LinearTransform"):
    super(LinearTransform, self).__init__(self,name=name)

    self.conv_r=Conv2D(1,kernel_size=3,strides=1,padding='same',use_bias=False)
    self.conv_g=Conv2D(1,kernel_size=3,strides=1,padding='same',use_bias=False)
    self.conv_b=Conv2D(1,kernel_size=3,strides=1,padding='same',use_bias=False)

    self.pool_rc=AveragePooling2D(pool_size=(patch_size,patch_size),strides=1)
    self.pool_gc=AveragePooling2D(pool_size=(patch_size,patch_size),strides=1)
    self.pool_bc=AveragePooling2D(pool_size=(patch_size,patch_size),strides=1)

    self.bn=BatchNormalization()
    self.sigmoid=Activation('sigmoid')
    self.softmax=Activation('softmax')

  def call(self, input,training=True):
    r,g,b=input[:,:,:,0:1],input[:,:,:,1:2],input[:,:,:,2:3]

    rs=self.conv_r(r)
    gs=self.conv_g(g)
    bs=self.conv_r(b)

    rc=tf.reshape(self.pool_rc(rs),[-1,1])
    gc=tf.reshape(self.pool_gc(gs),[-1,1])
    bc=tf.reshape(self.pool_bc(bs),[-1,1])

    merge=Concatenate(axis=-1)([rc,gc,bc])
    merge=tf.expand_dims(merge,axis=1)
    merge=tf.expand_dims(merge,axis=1)
    merge=self.softmax(merge)
    merge=tf.repeat(merge,repeats=48,axis=2)
    merge=tf.repeat(merge,repeats=48,axis=1)

    r=r*(1+self.sigmoid(rs))
    g=g*(1+self.sigmoid(gs))
    b=b*(1+self.sigmoid(bs))

    output=self.bn(merge[:,:,:,0:1]*r+merge[:,:,:,1:2]*g+merge[:,:,:,2:3]*b,training=training)
    return output

class ResBlock(tf.keras.Model):
  def __init__(self,out_ch,residual_path=False,stride=1):
    super(ResBlock,self).__init__(self)
    self.residual_path=residual_path

    self.conv1=Conv2D(out_ch,kernel_size=3,strides=stride,padding='same', use_bias=False,data_format="channels_last")
    self.bn1=BatchNormalization()
    self.relu1=LeakyReLU()#Activation('leaky_relu')

    self.conv2=Conv2D(out_ch,kernel_size=3,strides=1,padding='same', use_bias=False,data_format="channels_last")
    self.bn2=BatchNormalization()

    if residual_path:
      self.conv_shortcut=Conv2D(out_ch,kernel_size=1,strides=stride,padding='same',use_bias=False)
      self.bn_shortcut=BatchNormalization()

    self.relu2=LeakyReLU()#Activation('leaky_relu')

  def call(self,x,training=True):
    xs=self.relu1(self.bn1(self.conv1(x),training=training))
    xs=self.bn2(self.conv2(xs),training=training)

    if self.residual_path:
      x=self.bn_shortcut(self.conv_shortcut(x),training=training)
    #print(x.shape,xs.shape)
    xs=x+xs
    return self.relu2(xs)

class Unet(tf.keras.Model):
  def __init__(self):
    super(Unet,self).__init__(self)
    self.conv_init=LinearTransform()
    self.resinit=ResBlock(16,residual_path=True)
    self.up_sample=UpSampling2D(size=(2,2),interpolation='bilinear')
    self.resup=ResBlock(32,residual_path=True)

    self.pool1=MaxPool2D(pool_size=(2,2))

    self.resblock_down1=ResBlock(64,residual_path=True)
    self.resblock_down11=ResBlock(64,residual_path=False)
    self.pool2=MaxPool2D(pool_size=(2,2))

    self.resblock_down2=ResBlock(128,residual_path=True)
    self.resblock_down21=ResBlock(128,residual_path=False)
    self.pool3=MaxPool2D(pool_size=(2,2))

    self.resblock_down3=ResBlock(256,residual_path=True)
    self.resblock_down31=ResBlock(256,residual_path=False)
    self.pool4=MaxPool2D(pool_size=(2,2))

    self.resblock=ResBlock(512,residual_path=True)

    self.unpool3=UpSampling2D(size=(2,2),interpolation='bilinear')
    self.resblock_up3=ResBlock(256,residual_path=True)
    self.resblock_up31=ResBlock(256,residual_path=False)

    self.unpool2=UpSampling2D(size=(2,2),interpolation='bilinear')
    self.resblock_up2=ResBlock(128,residual_path=True)
    self.resblock_up21=ResBlock(128,residual_path=False)

    self.unpool1=UpSampling2D(size=(2,2),interpolation='bilinear')
    self.resblock_up1=ResBlock(64,residual_path=True)

    self.unpool_final=UpSampling2D(size=(2,2),interpolation='bilinear')
    self.resblock2=ResBlock(32,residual_path=True)

    self.pool_final=MaxPool2D(pool_size=(2,2))
    self.resfinal=ResBlock(32)

    self.conv_final=Conv2D(1,kernel_size=1,strides=1,padding='same',use_bias=False)
    self.bn_final=BatchNormalization()
    self.act=Activation('sigmoid')

  def call(self,x,training=True):
    x_linear=self.conv_init(x,training=training)
    x=self.resinit(x_linear,training=training)
    x=self.up_sample(x)
    x=self.resup(x,training=training)

    stage1=self.pool1(x)
    stage1=self.resblock_down1(stage1,training=training)
    stage1=self.resblock_down11(stage1,training=training)

    stage2=self.pool2(stage1)
    stage2=self.resblock_down2(stage2,training=training)
    stage2=self.resblock_down21(stage2,training=training)

    stage3=self.pool3(stage2)
    stage3=self.resblock_down3(stage3,training=training)
    stage3=self.resblock_down31(stage3,training=training)

    stage4=self.pool4(stage3)
    stage4=self.resblock(stage4,training=training)

    stage3=Concatenate(axis=3)([stage3,self.unpool3(stage4)])
    stage3=self.resblock_up3(stage3,training=training)
    stage3=self.resblock_up31(stage3,training=training)

    stage2=Concatenate(axis=3)([stage2,self.unpool2(stage3)])
    stage2=self.resblock_up2(stage2,training=training)
    stage2=self.resblock_up21(stage2,training=training)

    stage1=Concatenate(axis=3)([stage1,self.unpool1(stage2)])
    stage1=self.resblock_up1(stage1,training=training)

    x=Concatenate(axis=3)([x,self.unpool_final(stage1)])
    x=self.resblock2(x,training=training)

    x=self.pool_final(x)
    x=self.resfinal(x,training=training)

    seg_result=self.act(self.bn_final(self.conv_final(x),training=training))

    return x_linear,seg_result
class Unet2(tf.keras.Model):
  def __init__(self):
    super(Unet2,self).__init__(self)
    self.conv_init=LinearTransform()
    #self.resinit=ResBlock(16,residual_path=True)
    self.resinit=Conv2D(16,kernel_size=3,strides=1,padding='same',use_bias=False)
    self.up_sample=UpSampling2D(size=(2,2),interpolation='bilinear')
    #self.resup=ResBlock(32,residual_path=True)
    self.resup=Conv2D(32,kernel_size=3,strides=1,padding='same',use_bias=False)

    self.pool1=MaxPool2D(pool_size=(2,2))

    #self.resblock_down1=ResBlock(64,residual_path=True)
    self.resblock_down1=Conv2D(64,kernel_size=3,strides=1,padding='same',use_bias=False)
    #self.resblock_down11=ResBlock(64,residual_path=False)
    self.resblock_down11=Conv2D(64,kernel_size=3,strides=1,padding='same',use_bias=False)
    self.pool2=MaxPool2D(pool_size=(2,2))

    #self.resblock_down2=ResBlock(128,residual_path=True)
    self.resblock_down2=Conv2D(128,kernel_size=3,strides=1,padding='same',use_bias=False)
    #self.resblock_down21=ResBlock(128,residual_path=False)
    self.resblock_down21=Conv2D(128,kernel_size=3,strides=1,padding='same',use_bias=False)
    self.pool3=MaxPool2D(pool_size=(2,2))

    #self.resblock_down3=ResBlock(256,residual_path=True)
    self.resblock_down3=Conv2D(256,kernel_size=3,strides=1,padding='same',use_bias=False)
    #self.resblock_down31=ResBlock(256,residual_path=False)
    self.resblock_down31=Conv2D(256,kernel_size=3,strides=1,padding='same',use_bias=False)
    self.pool4=MaxPool2D(pool_size=(2,2))

    #self.resblock=ResBlock(512,residual_path=True)
    self.resblock=Conv2D(512,kernel_size=3,strides=1,padding='same',use_bias=False)

    self.unpool3=UpSampling2D(size=(2,2),interpolation='bilinear')
    #self.resblock_up3=ResBlock(256,residual_path=True)
    self.resblock_up3=Conv2D(256,kernel_size=3,strides=1,padding='same',use_bias=False)
    #self.resblock_up31=ResBlock(256,residual_path=False)
    self.resblock_up31=Conv2D(256,kernel_size=3,strides=1,padding='same',use_bias=False)

    self.unpool2=UpSampling2D(size=(2,2),interpolation='bilinear')
    #self.resblock_up2=ResBlock(128,residual_path=True)
    self.resblock_up2=Conv2D(128,kernel_size=3,strides=1,padding='same',use_bias=False)
    #self.resblock_up21=ResBlock(128,residual_path=False)
    self.resblock_up21=Conv2D(128,kernel_size=3,strides=1,padding='same',use_bias=False)

    self.unpool1=UpSampling2D(size=(2,2),interpolation='bilinear')
    #self.resblock_up1=ResBlock(64,residual_path=True)
    self.resblock_up1=Conv2D(64,kernel_size=3,strides=1,padding='same',use_bias=False)

    self.unpool_final=UpSampling2D(size=(2,2),interpolation='bilinear')
    #self.resblock2=ResBlock(32,residual_path=True)
    self.resblock2=Conv2D(32,kernel_size=3,strides=1,padding='same',use_bias=False)

    self.pool_final=MaxPool2D(pool_size=(2,2))
    #self.resfinal=ResBlock(32)
    self.resfinal=Conv2D(32,kernel_size=3,strides=1,padding='same',use_bias=False)

    self.conv_final=Conv2D(1,kernel_size=1,strides=1,padding='same',use_bias=False)
    self.bn_final=BatchNormalization()
    self.act=Activation('sigmoid')

  def call(self,x,training=True):
    x_linear=self.conv_init(x,training=training)
    x=self.resinit(x_linear,training=training)
    x=self.up_sample(x)
    x=self.resup(x,training=training)

    stage1=self.pool1(x)
    stage1=self.resblock_down1(stage1,training=training)
    stage1=self.resblock_down11(stage1,training=training)

    stage2=self.pool2(stage1)
    stage2=self.resblock_down2(stage2,training=training)
    stage2=self.resblock_down21(stage2,training=training)

    stage3=self.pool3(stage2)
    stage3=self.resblock_down3(stage3,training=training)
    stage3=self.resblock_down31(stage3,training=training)

    stage4=self.pool4(stage3)
    stage4=self.resblock(stage4,training=training)

    stage3=Concatenate(axis=3)([stage3,self.unpool3(stage4)])
    stage3=self.resblock_up3(stage3,training=training)
    stage3=self.resblock_up31(stage3,training=training)

    stage2=Concatenate(axis=3)([stage2,self.unpool2(stage3)])
    stage2=self.resblock_up2(stage2,training=training)
    stage2=self.resblock_up21(stage2,training=training)

    stage1=Concatenate(axis=3)([stage1,self.unpool1(stage2)])
    stage1=self.resblock_up1(stage1,training=training)

    x=Concatenate(axis=3)([x,self.unpool_final(stage1)])
    x=self.resblock2(x,training=training)

    x=self.pool_final(x)
    x=self.resfinal(x,training=training)

    seg_result=self.act(self.bn_final(self.conv_final(x),training=training))

    return x_linear,seg_result

class Unet3(tf.keras.Model):
    def __init__(self):
        super(Unet3, self).__init__()

        # Define the encoder layers
        #self.encoder_conv_init=LinearTransform()
        self.encoder_conv_init=Conv2D(1, 3, activation='relu', padding='same')
        self.encoder_conv1 = Conv2D(32, 3, activation='relu', padding='same')
        self.encoder_pool1 = MaxPool2D(pool_size=(2, 2))
        self.encoder_conv2 = Conv2D(64, 3, activation='relu', padding='same')
        self.encoder_pool2 = MaxPool2D(pool_size=(2, 2))
        self.encoder_conv3 = Conv2D(128, 3, activation='relu', padding='same')
        self.encoder_pool3 = MaxPool2D(pool_size=(2, 2))

        # Define the bottleneck layer
        self.bottleneck_conv = Conv2D(512, 3, activation='relu', padding='same')

        # Define the decoder layers
        self.decoder_upsample1 = UpSampling2D(size=(2, 2))
        self.decoder_conv4 = Conv2D(128, 3, activation='relu', padding='same')
        self.decoder_upsample2 = UpSampling2D(size=(2, 2))
        self.decoder_conv5 = Conv2D(64, 3, activation='relu', padding='same')
        self.decoder_upsample3 = UpSampling2D(size=(2, 2))
        self.decoder_conv6 = Conv2D(32, 3, activation='relu', padding='same')
        self.decoder_output = Conv2D(1, 3, activation='sigmoid', padding='same')

    def call(self, x, training=True):
        # Encoder
        x_linear=self.encoder_conv_init(x, training=training)
        enc1 = self.encoder_conv1(x_linear, training=training)
        enc1_pool = self.encoder_pool1(enc1)
        enc2 = self.encoder_conv2(enc1_pool, training=training)
        enc2_pool = self.encoder_pool2(enc2)
        enc3 = self.encoder_conv3(enc2_pool, training=training)
        enc3_pool = self.encoder_pool3(enc3)

        # Bottleneck
        bottleneck = self.bottleneck_conv(enc3_pool, training=training)

        # Decoder
        dec1 = self.decoder_upsample1(bottleneck)
        dec1 = Concatenate()([dec1, enc3])
        dec1 = self.decoder_conv4(dec1, training=training)
        dec2 = self.decoder_upsample2(dec1)
        dec2 = Concatenate()([dec2, enc2])
        dec2 = self.decoder_conv5(dec2, training=training)
        dec3 = self.decoder_upsample3(dec2)
        dec3 = Concatenate()([dec3, enc1])
        dec3 = self.decoder_conv6(dec3, training=training)

        # Output
        seg_result = self.decoder_output(dec3, training=training)
        return x_linear,seg_result

class Unet4(tf.keras.Model):
    def __init__(self):
        super(Unet4, self).__init__()

        # Define the encoder layers
        self.encoder_conv_init=LinearTransform()
        #self.encoder_conv_init=Conv2D(1, 3, activation='relu', padding='same')
        self.encoder_conv1 = Conv2D(32, 3, activation='relu', padding='same')
        self.encoder_pool1 = MaxPool2D(pool_size=(2, 2))
        self.encoder_conv2 = Conv2D(64, 3, activation='relu', padding='same')
        self.encoder_pool2 = MaxPool2D(pool_size=(2, 2))
        self.encoder_conv3 = Conv2D(128, 3, activation='relu', padding='same')
        self.encoder_pool3 = MaxPool2D(pool_size=(2, 2))

        # Define the bottleneck layer
        self.bottleneck_conv = Conv2D(512, 3, activation='relu', padding='same')

        # Define the decoder layers
        self.decoder_upsample1 = UpSampling2D(size=(2, 2))
        self.decoder_conv4 = Conv2D(128, 3, activation='relu', padding='same')
        self.decoder_upsample2 = UpSampling2D(size=(2, 2))
        self.decoder_conv5 = Conv2D(64, 3, activation='relu', padding='same')
        self.decoder_upsample3 = UpSampling2D(size=(2, 2))
        self.decoder_conv6 = Conv2D(32, 3, activation='relu', padding='same')
        self.decoder_output = Conv2D(1, 3, activation='sigmoid', padding='same')

    def call(self, x, training=True):
        # Encoder
        x_linear=self.encoder_conv_init(x, training=training)
        enc1 = self.encoder_conv1(x_linear, training=training)
        enc1_pool = self.encoder_pool1(enc1)
        enc2 = self.encoder_conv2(enc1_pool, training=training)
        enc2_pool = self.encoder_pool2(enc2)
        enc3 = self.encoder_conv3(enc2_pool, training=training)
        enc3_pool = self.encoder_pool3(enc3)

        # Bottleneck
        bottleneck = self.bottleneck_conv(enc3_pool, training=training)

        # Decoder
        dec1 = self.decoder_upsample1(bottleneck)
        dec1 = Concatenate()([dec1, enc3])
        dec1 = self.decoder_conv4(dec1, training=training)
        dec2 = self.decoder_upsample2(dec1)
        dec2 = Concatenate()([dec2, enc2])
        dec2 = self.decoder_conv5(dec2, training=training)
        dec3 = self.decoder_upsample3(dec2)
        dec3 = Concatenate()([dec3, enc1])
        dec3 = self.decoder_conv6(dec3, training=training)

        # Output
        seg_result = self.decoder_output(dec3, training=training)
        return x_linear,seg_result

class Unet5(tf.keras.Model):
    def __init__(self):
        super(Unet5, self).__init__()

        # Define the encoder layers
        #self.encoder_conv_init=LinearTransform()
        self.encoder_conv_init=Conv2D(1, 3, activation='relu', padding='same')
        #self.encoder_conv1 = Conv2D(32, 3, activation='relu', padding='same')
        self.encoder_conv1 = ResBlock(32,residual_path=True)
        self.encoder_pool1 = MaxPool2D(pool_size=(2, 2))
        #self.encoder_conv2 = Conv2D(64, 3, activation='relu', padding='same')
        self.encoder_conv2 = ResBlock(64,residual_path=True)
        self.encoder_pool2 = MaxPool2D(pool_size=(2, 2))
        #self.encoder_conv3 = Conv2D(128, 3, activation='relu', padding='same')
        self.encoder_conv3 = ResBlock(128,residual_path=True)
        self.encoder_pool3 = MaxPool2D(pool_size=(2, 2))

        # Define the bottleneck layer
        #self.bottleneck_conv = Conv2D(512, 3, activation='relu', padding='same')
        self.bottleneck_conv = ResBlock(512,residual_path=True)

        # Define the decoder layers
        self.decoder_upsample1 = UpSampling2D(size=(2, 2))
        #self.decoder_conv4 = Conv2D(128, 3, activation='relu', padding='same')
        self.decoder_conv4 = ResBlock(128,residual_path=True)
        self.decoder_upsample2 = UpSampling2D(size=(2, 2))
        #self.decoder_conv5 = Conv2D(64, 3, activation='relu', padding='same')
        self.decoder_conv5 = ResBlock(64,residual_path=True)
        self.decoder_upsample3 = UpSampling2D(size=(2, 2))
        #self.decoder_conv6 = Conv2D(32, 3, activation='relu', padding='same')
        self.decoder_conv6 = ResBlock(32,residual_path=True)
        #self.decoder_output = Conv2D(1, 3, activation='sigmoid', padding='same')
        self.decoder_output = ResBlock(1,residual_path=True)

    def call(self, x, training=True):
        # Encoder
        x_linear=self.encoder_conv_init(x, training=training)
        enc1 = self.encoder_conv1(x_linear, training=training)
        enc1_pool = self.encoder_pool1(enc1)
        enc2 = self.encoder_conv2(enc1_pool, training=training)
        enc2_pool = self.encoder_pool2(enc2)
        enc3 = self.encoder_conv3(enc2_pool, training=training)
        enc3_pool = self.encoder_pool3(enc3)

        # Bottleneck
        bottleneck = self.bottleneck_conv(enc3_pool, training=training)

        # Decoder
        dec1 = self.decoder_upsample1(bottleneck)
        dec1 = Concatenate()([dec1, enc3])
        dec1 = self.decoder_conv4(dec1, training=training)
        dec2 = self.decoder_upsample2(dec1)
        dec2 = Concatenate()([dec2, enc2])
        dec2 = self.decoder_conv5(dec2, training=training)
        dec3 = self.decoder_upsample3(dec2)
        dec3 = Concatenate()([dec3, enc1])
        dec3 = self.decoder_conv6(dec3, training=training)

        # Output
        seg_result = self.decoder_output(dec3, training=training)
        return x_linear,seg_result

# Define the ShuffleNet block as a custom layer
class ShuffleNetBlock(Layer):
    def __init__(self, out_channels, groups=4, **kwargs):
        super(ShuffleNetBlock, self).__init__(**kwargs)
        self.out_channels = out_channels
        self.groups = groups
    
    def build(self, input_shape):
        if self.out_channels % self.groups != 0:
            raise ValueError("The number of filters must be evenly divisible by the number of groups.")
    
        self.filters_per_group = self.out_channels // self.groups
        super().build(input_shape)
    
    def call(self, inputs):
        # 1x1 Convolution
        x = DepthwiseConv2D((3, 3), padding='same', activation='relu')(inputs)
        x = Conv2D(self.filters_per_group, (1, 1), activation='relu', padding='same')(x)
    
        # Grouped Convolution
        splits = tf.split(x, num_or_size_splits=self.groups, axis=-1)
        group_outputs = []
        for i in range(self.groups):
            group = Conv2D(self.filters_per_group, (3, 3), activation='relu', padding='same')(splits[i])
            group_outputs.append(group)
    
        # Concatenate the group outputs
        x = concatenate(group_outputs, axis=-1)
    
        # 1x1 Convolution to adjust the number of channels
        x = Conv2D(self.out_channels, (1, 1), activation='relu', padding='same')(x)
    
        return x


# Define the U-Net model with ShuffleNet blocks as a custom model
class Unet6(Model):
    def __init__(self, input_shape=(128, 128, 3), num_classes=1):
        super(Unet6, self).__init__()
        
        # Encoder
        self.conv0 = Conv2D(1, 3, activation='relu', padding='same')
        self.conv1 = ShuffleNetBlock(64)
        self.pool1 = MaxPooling2D(pool_size=(2, 2))
        self.conv2 = ShuffleNetBlock(128)
        self.pool2 = MaxPooling2D(pool_size=(2, 2))
        self.conv3 = ShuffleNetBlock(256)
        self.pool3 = MaxPooling2D(pool_size=(2, 2))
        
        # Bottleneck
        self.bottleneck = ShuffleNetBlock(512)
        
        # Decoder
        self.up4 = UpSampling2D(size=(2, 2))
        self.concat4 = concatenate
        self.conv4 = ShuffleNetBlock(256)
        self.up5 = UpSampling2D(size=(2, 2))
        self.concat5 = concatenate
        self.conv5 = ShuffleNetBlock(128)
        self.up6 = UpSampling2D(size=(2, 2))
        self.concat6 = concatenate
        self.conv6 = ShuffleNetBlock(64)

        # Output layer
        self.output_layer = Conv2D(num_classes, (1, 1), activation='sigmoid')
    
    def call(self, inputs):
        # Encoder
        conv0 = self.conv0(inputs)
        conv1 = self.conv1(conv0)
        pool1 = self.pool1(conv1)
        conv2 = self.conv2(pool1)
        pool2 = self.pool2(conv2)
        conv3 = self.conv3(pool2)
        pool3 = self.pool3(conv3)
        
        # Bottleneck
        bottleneck = self.bottleneck(pool3)
        
        # Decoder
        up4 = self.up4(bottleneck)
        concat4 = self.concat4([conv3, up4], axis=-1)
        conv4 = self.conv4(concat4)
        up5 = self.up5(conv4)
        concat5 = self.concat5([conv2, up5], axis=-1)
        conv5 = self.conv5(concat5)
        up6 = self.up6(conv5)
        concat6 = self.concat6([conv1, up6], axis=-1)
        conv6 = self.conv6(concat6)
        
        # Output layer
        output = self.output_layer(conv6)
        
        return conv0, output


class custom1(tf.keras.Model):
    def __init__(self):
        super(custom1, self).__init__()

        # Define the encoder layers
        #self.encoder_conv_init=Conv2D(1, 3, activation='relu', padding='same')
        self.encoder_conv_init=LinearTransform()
        #self.encoder_conv1 = ResBlock(32,residual_path=True)
        self.encoder_conv1 = Conv2D(64, 3, activation='relu', padding='same')
        #self.encoder_conv2 = ResBlock(64,residual_path=True)
        self.encoder_conv2 = Conv2D(64, 3, activation='relu', padding='same')
        self.encoder_conv3 = Conv2DTranspose(filters=32, kernel_size=(3, 3), strides=(2, 2), padding='same')
        self.encoder_conv4 = Conv2D(32, 3, activation='relu', padding='same')
        self.encoder_conv5 = Conv2D(32, 3, activation='relu', padding='same')
        self.encoder_conv6 = Conv2D(64, 3, (2, 2), activation='relu', padding='same')
        self.encoder_conv7 = Conv2D(64, 3, activation='relu', padding='same')
        #self.encoder_conv8 = ResBlock(64,residual_path=True)
        self.encoder_conv8 = Conv2D(64, 3, activation='relu', padding='same')
        self.encoder_conv9 = Conv2D(128, 3, (2, 2), activation='relu', padding='same')
        self.encoder_conv10 = Conv2D(128, 3, activation='relu', padding='same')
        self.encoder_conv11 = Conv2D(128, 3, activation='relu', padding='same')
        self.encoder_conv12 = Conv2DTranspose(filters=64, kernel_size=(3, 3), strides=(2, 2), padding='same')
        self.encoder_conv13 = Conv2D(32, 3, activation='relu', padding='same')
        self.encoder_conv14 = Conv2D(32, 3, activation='relu', padding='same')
        self.encoder_conv15 = Conv2D(1, 1, activation='relu', padding='same')   
        self.final = Activation('tanh')

    def call(self, x, training=True):
        # Encoder
        x_linear=self.encoder_conv_init(x, training=training)
        ed = self.encoder_conv1(x_linear, training=training)
        eds1 = self.encoder_conv2(ed, training=training)
        ed = self.encoder_conv3(eds1, training=training)
        ed = self.encoder_conv4(ed, training=training)
        ed = self.encoder_conv5(ed, training=training)
        ed = self.encoder_conv6(ed, training=training)
        ed = Concatenate(axis=3)([ed,eds1])
        ed = self.encoder_conv7(ed, training=training)
        eds2 = self.encoder_conv8(ed, training=training)
        ed = self.encoder_conv9(eds2, training=training)
        ed = self.encoder_conv10(ed, training=training)
        ed = self.encoder_conv11(ed, training=training)
        ed = self.encoder_conv12(ed, training=training)
        ed = Concatenate(axis=3)([ed,eds2])
        ed = self.encoder_conv13(ed, training=training)
        ed = self.encoder_conv14(ed, training=training)
        ed = self.encoder_conv15(ed, training=training)
        seg_result = self.final(ed, training=training)


        return x_linear,seg_result

class custom2(tf.keras.Model):
  def __init__(self):
    super(custom2,self).__init__(self)
    self.conv_init=LinearTransform()#128*128*1
    self.convd111=Conv2D(3,kernel_size=(3, 3),strides=(1, 1),padding='same',activation='relu', data_format="channels_last")#128*128*3
    self.convd1=Conv2D(64,kernel_size=(3, 3),strides=(1, 1),padding='same',activation='relu', data_format="channels_last")#128*128*64
    self.lcm11d1=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm12d1=Conv2D(64,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")
    self.mpd1=MaxPool2D(pool_size=(2,2))#64*64*64

    self.lcm11d2=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm12d2=Conv2D(128,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")#64*64*128
    self.lcm21d2=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm22d2=Conv2D(128,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")
    self.mpd2=MaxPool2D(pool_size=(2,2))#32*32*128

    self.lcm11d3=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm12d3=Conv2D(256,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")#64*64*256
    self.lcm21d3=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm22d3=Conv2D(256,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")#64*64*256
    self.mpd3=MaxPool2D(pool_size=(2,2))#32*32*256

    self.lcm11d4=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm12d4=Conv2D(512,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")#32*32*512
    self.lcm21d4=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm22d4=Conv2D(512,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")#32*32*512
    self.mpd4=MaxPool2D(pool_size=(2,2))#16*16*512

    self.lcm11d5=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm12d5=Conv2D(512,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")
    self.lcm21d5=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm22d5=Conv2D(512,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")
    self.mpd5=MaxPool2D(pool_size=(2,2))#16*16*512

    self.lcm11d6=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm12d6=Conv2D(1024,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")#16*16*1024
    self.lcm21d6=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm22d6=Conv2D(1024,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")
    self.mpd6=MaxPool2D(pool_size=(2,2))#8*8*1024

    self.upe1=UpSampling2D(size=(2,2),interpolation='bilinear')#16*16*1024
    self.lcm11e1=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm12e1=Conv2D(512,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")#16*16*512
    self.lcm21e1=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm22e1=Conv2D(512,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")#16*16*512

    self.upe2=UpSampling2D(size=(2,2),interpolation='bilinear')
    self.lcm11e2=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm12e2=Conv2D(512,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")
    self.lcm21e2=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm22e2=Conv2D(512,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")

    self.upe3=UpSampling2D(size=(2,2),interpolation='bilinear')
    self.lcm11e3=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm12e3=Conv2D(256,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")
    self.lcm21e3=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm22e3=Conv2D(256,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")

    self.upe4=UpSampling2D(size=(2,2),interpolation='bilinear')
    self.lcm11e4=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm12e4=Conv2D(128,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")
    self.lcm21e4=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm22e4=Conv2D(128,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")

    self.upe5=UpSampling2D(size=(2,2),interpolation='bilinear')
    self.lcm11e5=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm12e5=Conv2D(64,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")
    self.lcm21e5=DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same')
    self.lcm22e5=Conv2D(64,kernel_size=(1, 1),strides=1,padding='same', use_bias=False,data_format="channels_last")

    self.out=Conv2D(1,kernel_size=(1, 1),strides=1,padding='same',activation='softmax', use_bias=False,data_format="channels_last")


  def call(self,x,training=True):
    #x_linear = self.conv_init(x, training=training)
    #x_l2=self.convd111(x_linear, training=training)
    x_linear=self.convd1(x, training=training)#128*128*64
    x1d1=self.lcm11d1(x_linear, training=training)
    x2d1=self.lcm12d1(x1d1, training=training)#128*128*64
    x3d1=self.mpd1(x2d1)

    x1d2=self.lcm11d2(x3d1, training=training)
    x2d2=self.lcm12d2(x1d2, training=training)#64*64*128
    x3d2=self.lcm21d2(x2d2, training=training)
    x4d2=self.lcm22d2(x3d2, training=training)
    x5d2=self.mpd2(x4d2)

    x1d3=self.lcm11d3(x5d2, training=training)
    x2d3=self.lcm12d3(x1d3, training=training)#32*32*256
    x3d3=self.lcm21d3(x2d3, training=training)
    x4d3=self.lcm22d3(x3d3, training=training)
    x5d3=self.mpd3(x4d3)

    x1d4=self.lcm11d4(x5d3, training=training)
    x2d4=self.lcm12d4(x1d4, training=training)#16*16*512
    x3d4=self.lcm21d4(x2d4, training=training)
    x4d4=self.lcm22d4(x3d4, training=training)
    x5d4=self.mpd4(x4d4)

    x1d5=self.lcm11d5(x5d4, training=training)
    x2d5=self.lcm12d5(x1d5, training=training)#8*8*512
    x3d5=self.lcm21d5(x2d5, training=training)
    x4d5=self.lcm22d5(x3d5, training=training)
    x5d5=self.mpd5(x4d5)

    x1d6=self.lcm11d6(x5d5, training=training)
    x2d6=self.lcm12d6(x1d6, training=training)#4*4*1024
    x3d6=self.lcm21d6(x2d6, training=training)
    x4d6=self.lcm22d6(x3d6, training=training)

    x1e1=self.upe1(x4d6)
    x2e1=self.lcm11e1(x1e1, training=training)
    x3e1=self.lcm12e1(x2e1, training=training)#8*8*512
    x4e1=self.lcm21e1(x3e1, training=training)
    x5e1=self.lcm22e1(x4e1, training=training)
    x6e1=Concatenate(axis=-1)([x5e1,x4d5])

    x1e2=self.upe2(x6e1, training=training)
    x2e2=self.lcm11e2(x1e2, training=training)
    x3e2=self.lcm12e2(x2e2, training=training)#16*16
    x4e2=self.lcm21e2(x3e2, training=training)
    x5e2=self.lcm22e2(x4e2, training=training)
    x6e2=Concatenate(axis=-1)([x5e2,x4d4])

    x1e3=self.upe3(x6e2, training=training)
    x2e3=self.lcm11e3(x1e3, training=training)
    x3e3=self.lcm12e3(x2e3, training=training)#32*32
    x4e3=self.lcm21e3(x3e3, training=training)
    x5e3=self.lcm22e3(x4e3, training=training)
    x6e3=Concatenate(axis=-1)([x5e3,x4d3])

    x1e4=self.upe4(x6e3, training=training)
    x2e4=self.lcm11e4(x1e4, training=training)
    x3e4=self.lcm12e4(x2e4, training=training)#64*64
    x4e4=self.lcm21e4(x3e4, training=training)
    x5e4=self.lcm22e4(x4e4, training=training)
    x6e5=Concatenate(axis=-1)([x5e4,x4d2])

    x1e5=self.upe5(x6e5, training=training)
    x2e5=self.lcm11e5(x1e5, training=training)
    x3e5=self.lcm12e5(x2e5, training=training)#128*128
    x4e5=self.lcm21e5(x3e5, training=training)
    x5e5=self.lcm22e5(x4e5, training=training)
    x6e5=Concatenate(axis=-1)([x5e5,x2d1])

    seg_result=self.out(x6e5, training=training)

    return x_linear,seg_result

cols = ["#", "patch_size", "patch_num", "patch_threshold", "batch_size", "learning_rate"]
rows = [[ "value", patch_size, patch_num, patch_threshold, BATCH_SIZE, LR],
        ["comment", "(48*48) windows", "number of windows", "threshold for the patch, the smaller threshoold, the less vessel in the patch", "batch", "LR"]]

print(tabulate(rows, headers=cols,tablefmt="fancy_grid"))

ModuleNotFoundError: No module named 'tabulate'

In [3]:
#model_unet1=Unet()
#model_unet2=Unet2()
#model_unet3=Unet3()
#model_unet4=Unet4()
#model_unet5=Unet5()
#model_custom1=custom1()
#model_custom2=custom2()
#linear_output1, seg_result1 = model_unet1(input_tensor)
#linear_output2, seg_result2 = model_unet2(input_tensor)
#linear_output3, seg_result3 = model_unet3(input_tensor)
#linear_output4, seg_result4 = model_unet4(input_tensor)
#linear_output5, seg_result5 = model_unet5(input_tensor)
#linear_outputc1, seg_resultc1 = model_custom1(input_tensor)
#linear_outputc2, seg_resultc2 = model_custom2(input_tensor)
#model.summary(line_length=110)
#model_unet1.count_params()
#model_unet2.count_params()
#model_unet3.count_params()

#Unet1 binary crossentropy loss
train_loss_bc=0.005448818;train_acc_bc=0.9918593;train_f1_bc=0.6594059;train_sp_bc=0.9915452;train_se_bc=0.9915452;train_precision_bc=0.9914305;train_auroc_bc=0.9915449
val_loss_bc=0.28848;val_acc_bc=0.85171;val_f1_bc=0.53667;val_sp_bc=0.90131;val_se_bc=0.90131;val_precision_bc=0.90225;val_auroc_bc=0.90130
trained_till_epoch_bc=120; lowest_val_loss_on_epoch_bc=44; highest_val_acc_on_epoch_bc=96; highest_val_f1_on_epoch_bc=96; highest_val_sp_on_epoch_bc=96; highest_val_se_on_epoch_bc=96;highest_val_prec_on_epoch_bc=96;highest_val_auroc_on_epoch_bc=96
#Unet1 dice loss 
train_loss_d=0.0782; train_acc_d=0.9218; train_f1_d=0.5979; train_sp_d=0.8926; train_se_d=0.8926; train_precision_d=0.8920; train_auroc_d=0.8926
val_loss_d=0.1992; val_acc_d=0.8008; val_f1_d=0.4933; val_sp_d=0.8016; val_se_d=0.8016; val_precision_d=0.8011; val_auroc_d=0.8016
trained_till_epoch_d="Nan"; lowest_val_loss_on_epoch_d="Nan"
# Unet1 focal loss
train_loss_f=170.6789; train_acc_f=0.8527; train_f1_f=0.5390; train_sp_f=0.7866; train_se_f=0.7040; train_precision_f=0.7859; train_auroc_f=0.7866
val_loss_f=4232.2256; val_acc_f=0.7367; val_f1_f=0.4396; val_sp_f=0.7040; val_se_f=0.8016; val_precision_f=0.7028; val_auroc_f=0.7040
trained_till_epoch_f="Nan"; lowest_val_loss_on_epoch_f="Nan"
# Unet 2 binary crossentropy loss
train_loss_u2_bc=0.0693; train_acc_u2_bc=0.8881; train_f1_u2_bc=0.5682; train_sp_u2_bc=0.8840; train_se_u2_bc=0.8840; train_precision_u2_bc=0.8833; train_auroc_u2_bc=0.8840
val_loss_u2_bc=0.2361; val_acc_u2_bc=0.7802; val_f1_u2_bc=0.4762; val_sp_u2_bc=0.8205; val_se_u2_bc=0.8205; val_precision_u2_bc=0.8201; val_auroc_u2_bc=0.8205
trained_till_epoch_u2_bc=71; lowest_val_loss_on_epoch_u2_bc=33;
# Unet 3 binary crossentropy loss
train_loss_u3_bc=0.09351323; train_acc_u3_bc=0.8605181; train_f1_u3_bc=0.5424623; train_sp_u3_bc=0.86103207; train_se_u3_bc=0.86103207; train_precision_u3_bc=0.860599; train_auroc_u3_bc=0.86102957
val_loss_u3_bc=0.15823; val_acc_u3_bc=0.83833; val_f1_u3_bc=0.52537; val_sp_u3_bc=0.85357; val_se_u3_bc=0.85357; val_precision_u3_bc=0.85452; val_auroc_u3_bc=0.85358
trained_till_epoch_u3_bc=131; lowest_val_loss_on_epoch_u3_bc=29;highest_val_acc_on_epoch_u3_bc=68;highest_val_f1_on_epoch_u3_bc=68;highest_val_sp_on_epoch_u3_bc=109;highest_val_se_on_epoch_u3_bc=109;highest_val_prec_on_epoch_u3_bc=109;highest_val_auroc_on_epoch_u3_bc=109;
# Unet 4 binary crossentropy loss
train_loss_u4_bc=0.0733392; train_acc_u4_bc=0.88768107; train_f1_u4_bc=0.567843; train_sp_u4_bc=0.88820463; train_se_u4_bc=0.88820463; train_precision_u4_bc=0.88785917; train_auroc_u4_bc=0.8882016
val_loss_u4_bc=0.14747; val_acc_u4_bc=0.83952; val_f1_u4_bc=0.52617; val_sp_u4_bc=0.86361; val_se_u4_bc=0.86361; val_precision_u4_bc=0.86396; val_auroc_u4_bc=0.86361
trained_till_epoch_u4_bc=35; lowest_val_loss_on_epoch_u4_bc=6; highest_val_acc_on_epoch_u4_bc=30; highest_val_f1_on_epoch_u4_bc=30; highest_val_sp_on_epoch_u4_bc=27; highest_val_se_on_epoch_u4_bc=27;highest_val_prec_on_epoch_u4_bc=27;highest_val_auroc_on_epoch_u4_bc=27

# Unet 5 binary crossentropy loss
train_loss_u5_bc=0.17507; train_acc_u5_bc=0.74864; train_f1_u5_bc=0.44906; train_sp_u5_bc=0.74322; train_se_u5_bc=0.74322; train_precision_u5_bc=0.74317; train_auroc_u5_bc=0.74322
val_loss_u5_bc=0.15751; val_acc_u5_bc=0.7973; val_f1_u5_bc=0.49042; val_sp_u5_bc=0.85111; val_se_u5_bc=0.85111; val_precision_u5_bc=0.8576555; val_auroc_u5_bc=0.85112
trained_till_epoch_u5_bc=45; lowest_val_loss_on_epoch_u5_bc=31; highest_val_acc_on_epoch_u5_bc=30; highest_val_f1_on_epoch_u5_bc=30; highest_val_sp_on_epoch_u5_bc=1; highest_val_se_on_epoch_u5_bc=1;highest_val_prec_on_epoch_u5_bc=1;highest_val_auroc_on_epoch_u5_bc=1

# Custom net (sine-net) patch 48, patch_num 1500
train_loss_c1_bc=0.0811179; train_acc_c1_bc=0.8779; train_f1_c1_bc=0.5594; train_sp_c1_bc=0.8773; train_se_c1_bc=0.8773; train_precision_c1_bc=0.8768; train_auroc_c1_bc=0.8773
val_loss_c1_bc=0.14179; val_acc_c1_bc=0.832317; val_f1_c1_bc=0.520198; val_sp_c1_bc=0.85573; val_se_c1_bc=0.85573; val_precision_c1_bc=0.856372; val_auroc_c1_bc=0.855734
trained_till_epoch_c1_bc=141; lowest_val_loss_on_epoch_c1_bc=90; highest_val_acc_on_epoch_c1_bc=46; highest_val_f1_on_epoch_c1_bc=46; highest_val_sp_on_epoch_c1_bc=87; highest_val_se_on_epoch_c1_bc=87;highest_val_prec_on_epoch_c1_bc=87;highest_val_auroc_on_epoch_c1_bc=87


# Custom2 net patch 128, patch_num 1500
train_loss_c2_bc=0.14007; train_acc_c2_bc=0.25734; train_f1_c2_bc=0.13753; train_sp_c2_bc=0.14771; train_se_c2_bc=0.14771; train_precision_c2_bc=0.14771; train_auroc_c2_bc=0.14771
val_loss_c2_bc=0.12809; val_acc_c2_bc=0.30818; val_f1_c2_bc=0.16695; val_sp_c2_bc=0.18216; val_se_c2_bc=0.18216; val_precision_c2_bc=0.18216; val_auroc_c2_bc=0.18216
trained_till_epoch_c2_bc=41; lowest_val_loss_on_epoch_c2_bc=39; highest_val_acc_on_epoch_c2_bc=2; highest_val_f1_on_epoch_c2_bc=2; highest_val_sp_on_epoch_c2_bc=2; highest_val_se_on_epoch_c2_bc=2;highest_val_prec_on_epoch_c2_bc=2;highest_val_auroc_on_epoch_c2_bc=2


# info about nets
x = PrettyTable()
x.field_names = [f"Net Config", "Net", "Number of parameters", "comments"]
x.add_row(["report", "UNet1", "11,351,706", "The Unet architecture which uses residual blocks and residual path and skip connections"])
x.add_row(["report", "UNet2", "5,442,250", "The same UNet, but replacing all the res blocks with Conv2D"])
x.add_row(["report", "UNet3", "1,559,069", "This is a basic UNet model without even the linear transform for preprocessing"])
x.add_row(["report", "UNet4", "1,559,063", "The same basic UNet with linear transform for preprocessing"])
x.add_row(["report", "UNet5", "4,489,137", "This is the UNet3 plus residual blocks defined in UNet1 instead of conv blocks"])
x.add_row(["report", "custom1", "692,759", "sine-net paper network but with patch_size 48 and patch_num 1500"])
x.add_row(["report", "custom2", "4,715,008", "fast and efficient rbvs paper network  with patch_size 128. In the paper parameters are reported to be 10M"])
print(x)

# train
x = PrettyTable()
x.field_names = [f"Metrics train data", "Net", "loss_function", "train_loss", "train_acc", "train_f1", "train_specificity", "train_sensitivity", "train_precision", "train_auroc"]
x.add_row([f"report on epoch {trained_till_epoch_bc}", "UNet1", "binary_crossentropy", train_loss_bc, train_acc_bc, train_f1_bc, train_sp_bc, train_se_bc, train_precision_bc, train_auroc_bc])
x.add_row([f"report on epoch {trained_till_epoch_d}", "UNet1", "dice_loss", train_loss_d, train_acc_d, train_f1_d, train_sp_d, train_se_d, train_precision_d, train_auroc_d])
x.add_row([f"report on epoch {trained_till_epoch_f}", "UNet1", "focal_loss", train_loss_f, train_acc_f, train_f1_f, train_sp_f, train_se_f, train_precision_f, train_auroc_f])
x.add_row([f"report on epoch {trained_till_epoch_u2_bc}", "UNet2", "binary_crossentropy", train_loss_u2_bc, train_acc_u2_bc, train_f1_u2_bc, train_sp_u2_bc, train_se_u2_bc, train_precision_u2_bc, train_auroc_u2_bc])
x.add_row([f"report on epoch {trained_till_epoch_u3_bc}", "UNet3", "binary_crossentropy", train_loss_u3_bc, train_acc_u3_bc, train_f1_u3_bc, train_sp_u3_bc, train_se_u3_bc, train_precision_u3_bc, train_auroc_u3_bc])
x.add_row([f"report on epoch {trained_till_epoch_u4_bc}", "UNet4", "binary_crossentropy", train_loss_u4_bc, train_acc_u4_bc, train_f1_u4_bc, train_sp_u4_bc, train_se_u4_bc, train_precision_u4_bc, train_auroc_u4_bc])
x.add_row([f"report on epoch {trained_till_epoch_u5_bc}", "UNet5", "binary_crossentropy", train_loss_u5_bc, train_acc_u5_bc, train_f1_u5_bc, train_sp_u5_bc, train_se_u5_bc, train_precision_u5_bc, train_auroc_u5_bc])
x.add_row([f"report on epoch {trained_till_epoch_c1_bc}", "custom1", "binary_crossentropy", train_loss_c1_bc, train_acc_c1_bc, train_f1_c1_bc, train_sp_c1_bc, train_se_c1_bc, train_precision_c1_bc, train_auroc_c1_bc])
x.add_row([f"report on epoch {trained_till_epoch_c1_bc}", "custom2", "binary_crossentropy", train_loss_c2_bc, train_acc_c2_bc, train_f1_c2_bc, train_sp_c2_bc, train_se_c2_bc, train_precision_c2_bc, train_auroc_c2_bc])
print(x)


# validation
x = PrettyTable()
x.field_names = [f"Metrics val data", "Net", "loss_function", "val_loss", "val_acc", "val_f1", "val_specificity", "val_sensitivity", "val_precision", "val_auroc"]
x.add_row([f"report on best epoch results", "UNet1", "binary_crossentropy", val_loss_bc, val_acc_bc, val_f1_bc, val_sp_bc, val_se_bc, val_precision_bc, val_auroc_bc])
x.add_row([f"report on epoch {trained_till_epoch_d}", "UNet1", "dice_loss", val_loss_d, val_acc_d, val_f1_d, val_sp_d, val_se_d, val_precision_d, val_auroc_d])
x.add_row([f"report on epoch {trained_till_epoch_f}", "UNet1", "focal_loss", val_loss_f, val_acc_f, val_f1_f, val_sp_f, val_se_f, val_precision_f, val_auroc_f])
x.add_row([f"report on epoch {trained_till_epoch_u2_bc}", "UNet2", "binary_crossentropy", val_loss_u2_bc, val_acc_u2_bc, val_f1_u2_bc, val_sp_u2_bc, val_se_u2_bc, val_precision_u2_bc, val_auroc_u2_bc])
x.add_row([f"report on best epoch results {trained_till_epoch_u3_bc}", "UNet3", "binary_crossentropy", val_loss_u3_bc, val_acc_u3_bc, val_f1_u3_bc, val_sp_u3_bc, val_se_u3_bc, val_precision_u3_bc, val_auroc_u3_bc])
x.add_row([f"report on best epoch results", "UNet4", "binary_crossentropy", val_loss_u4_bc, val_acc_u4_bc, val_f1_u4_bc, val_sp_u4_bc, val_se_u4_bc, val_precision_u4_bc, val_auroc_u4_bc])
x.add_row([f"report on best epoch results", "UNet5", "binary_crossentropy", val_loss_u5_bc, val_acc_u5_bc, val_f1_u5_bc, val_sp_u5_bc, val_se_u5_bc, val_precision_u5_bc, val_auroc_u5_bc])
x.add_row([f"report on best epoch results", "custom1", "binary_crossentropy", val_loss_c1_bc, val_acc_c1_bc, val_f1_c1_bc, val_sp_c1_bc, val_se_c1_bc, val_precision_c1_bc, val_auroc_c1_bc])
x.add_row([f"report on best epoch results", "custom2", "binary_crossentropy", val_loss_c2_bc, val_acc_c2_bc, val_f1_c2_bc, val_sp_c2_bc, val_se_c2_bc, val_precision_c2_bc, val_auroc_c2_bc])
print(x)

# epochs info
x = PrettyTable()
x.field_names = [f"val metrics changes on epochs", "trained epoch", "lowest val_loss epoch", "highest val_acc epoch", "highest val_f1 epoch", "highest val_sp epoch", "highest val_se epoch", "highest val_prec epoch", "highest val_auroc epoch"]
x.add_row(["UNet1_binarycrossentropy", trained_till_epoch_bc, lowest_val_loss_on_epoch_bc, highest_val_acc_on_epoch_bc, highest_val_f1_on_epoch_bc, highest_val_sp_on_epoch_bc, highest_val_se_on_epoch_bc, highest_val_prec_on_epoch_bc, highest_val_auroc_on_epoch_bc])
x.add_row(["UNet1_dice_loss", trained_till_epoch_d, lowest_val_loss_on_epoch_d, "NaN", "NaN", "NaN", "NaN", "NaN", "NaN"])
x.add_row(["UNet1_focal_loss", trained_till_epoch_f, lowest_val_loss_on_epoch_f, "NaN", "NaN", "NaN", "NaN", "NaN", "NaN"])
x.add_row(["UNet2_binarycrossentropy", trained_till_epoch_u2_bc, lowest_val_loss_on_epoch_u2_bc, "NaN", "NaN", "NaN", "NaN", "NaN", "NaN"])
x.add_row(["UNet3_binarycrossentropy", trained_till_epoch_u3_bc, lowest_val_loss_on_epoch_u3_bc, highest_val_acc_on_epoch_u3_bc, highest_val_f1_on_epoch_u3_bc, highest_val_sp_on_epoch_u3_bc, highest_val_se_on_epoch_u3_bc, highest_val_prec_on_epoch_u3_bc, highest_val_auroc_on_epoch_u3_bc])
x.add_row(["UNet4_binarycrossentropy", trained_till_epoch_u4_bc, lowest_val_loss_on_epoch_u4_bc, highest_val_acc_on_epoch_u4_bc, highest_val_f1_on_epoch_u4_bc, highest_val_sp_on_epoch_u4_bc, highest_val_se_on_epoch_u4_bc, highest_val_prec_on_epoch_u4_bc, highest_val_auroc_on_epoch_u4_bc])
x.add_row(["UNet5_binarycrossentropy", trained_till_epoch_u5_bc, lowest_val_loss_on_epoch_u5_bc, highest_val_acc_on_epoch_u5_bc, highest_val_f1_on_epoch_u5_bc, highest_val_sp_on_epoch_u5_bc, highest_val_se_on_epoch_u5_bc, highest_val_prec_on_epoch_u5_bc, highest_val_auroc_on_epoch_u5_bc])
x.add_row(["custom1_binarycrossentropy", trained_till_epoch_c1_bc, lowest_val_loss_on_epoch_c1_bc, highest_val_acc_on_epoch_c1_bc, highest_val_f1_on_epoch_c1_bc, highest_val_sp_on_epoch_c1_bc, highest_val_se_on_epoch_c1_bc, highest_val_prec_on_epoch_c1_bc, highest_val_auroc_on_epoch_c1_bc])
x.add_row(["custom2_binarycrossentropy", trained_till_epoch_c2_bc, lowest_val_loss_on_epoch_c2_bc, highest_val_acc_on_epoch_c2_bc, highest_val_f1_on_epoch_c2_bc, highest_val_sp_on_epoch_c2_bc, highest_val_se_on_epoch_c2_bc, highest_val_prec_on_epoch_c2_bc, highest_val_auroc_on_epoch_c2_bc])
print(x)


NameError: name 'Unet' is not defined